<a href="https://colab.research.google.com/github/GurkiratSarna/Information-Retrieval/blob/master/Assigment_1_1900690.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indexing for Web Search
## Information Retrieval Assignment 1
### Registeration ID's :
### 1900690
### 1900863

Installing and importing the required packages and libraries

In [0]:
# Umcomment and execute only once
#pip install spacy

In [0]:
# Umcomment and execute only once
#pip install inflect

In [0]:
import nltk, re
import sklearn
import urllib
import urllib.request
import urllib.parse
import numpy as np
import pandas as pd
import collections
import sys
from bs4 import BeautifulSoup
from nltk import sent_tokenize, word_tokenize
import inflect
import string, unicodedata
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import pos_tag
from nltk import ne_chunk
#from google.colab import files
import math

In [0]:
'''----Installing the nltk package----
please follow the prompted instructions after execution of this line of code for successfull download of nltk packages.
'''
nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

Below is the main cell which has to be executed to get the outputs.
Note : All the other cells need to be run before executing the below cell.

In [97]:
#Get web pages fromt he user
html_link=[]
html_links=get_webpages()

#Preprocessing to get the stems of all the web pages
stems=[]
for i in html_links:
  stems.append(preprocessing(i))

tag=[]
#POS Tagging
for i in stems:
  tag.append(pos_tagging(i))

#Entity Recognition
NER=[]
for i in tag:
  NER.append(entity_reco(i))

#Create a dictionary for all the webpages
var_dict=var_dictionary(html_links)

#Get the word count in each web page
var_dict=word_freq(var_dict,stems)

#Calculate Term Frequency
stem_list_index=0
TFBow={**var_dict}
#Call computeTF function to for each dictionary value.
for key, TFBowval in TFBow.items():
  TFBow[key]=computeTF(TFBowval,stems[stem_list_index])
  stem_list_index +=1

#For displaying properly as a data frame
test=pd.DataFrame(TFBow)
test=test.transpose()
display(test)

#Computing the IDF for both documents

#Create a list of dictionary variables
temp_list=[]
for k, v in var_dict.items():
  temp_list.append(var_dict[k])
idfs = computeIDF(temp_list)

#Calculating the TF.IDF score for all documents
tfidfBow={**TFBow}
for k, v in tfidfBow.items():
    tfidfBow[k]=computeTFIDF(v, idfs)


#Data frame of the scores
#For displaying properly as a data frame
score=pd.DataFrame(tfidfBow)
score=score.transpose()
display(score)

for key, val in tfidfBow.items():
  scorest=sort_score(val)
  #To download the final score files uncomment the below line, for every document a pop up will ask for the location where the file has to be stored.
  #download_file(scorest)


Please enter the web link : http://www.multimediaeval.org/mediaeval2019/memorability/
Do you wish to enter more html links ? (yes/NO) : yes
Please enter the web link : https://sites.google.com/view/siirh2020/
Do you wish to enter more html links ? (yes/NO) : 

 In the function HTML LIST : 
['http://www.multimediaeval.org/mediaeval2019/memorability/', 'https://sites.google.com/view/siirh2020/']


,10000,1159pm,14691482,16921703,2008videos,2009videoclef,2010videoclef,2011mediaeval,2012mediaeval,2013mediaeval,2014mediaeval,2015mediaeval,2016mediaeval,2017mediaeval,2018mediaeval,2019emotion,2019runs,2019test,2019working,2020camera,2020notificationsfull,2020short,23902398,2729,27302739,2931,42nd,abl,abstract,abus,abusep,acceiv,accompany,acm,addit,address,adity,adv,advert,aesthet,...,usathierry,usayonghu,usayoshinobu,usazit,valenc,vast,venu,via,video,videomediaev,videomem,view,vigo,villega,villetaneus,vis,vocab,vorh,way,webcont,wei,wel,wellbeingmedico,western,work,workshop,workshopa,workshopsearch,workshopsem,writ,wu,wwwspringercomgpcomputersciencelncsconferenceproceedingsguidelineseasychair,wwwtechnicolorcomdreamresearchinnovationmoviememorabilitydatasettask,xavy,xiao,xintao,yadat,year,zurich,ştef
wordDict0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,0,2,0,0,0,0,1,1,0,1,1,0,1,2,...,0,0,0,0,0,0,0,1,22,1,1,2,0,0,0,5,0,0,1,0,0,2,1,0,2,2,0,0,0,0,0,0,1,1,1,1,1,2,0,1
wordDict1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,4,1,1,1,0,0,1,0,0,1,0,0,...,1,1,1,1,1,1,2,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,7,1,1,1,1,1,1,0,0,0,0,0,0,1,0


,10000,1159pm,14691482,16921703,2008videos,2009videoclef,2010videoclef,2011mediaeval,2012mediaeval,2013mediaeval,2014mediaeval,2015mediaeval,2016mediaeval,2017mediaeval,2018mediaeval,2019emotion,2019runs,2019test,2019working,2020camera,2020notificationsfull,2020short,23902398,2729,27302739,2931,42nd,abl,abstract,abus,abusep,acceiv,accompany,acm,addit,address,adity,adv,advert,aesthet,...,usathierry,usayonghu,usayoshinobu,usazit,valenc,vast,venu,via,video,videomediaev,videomem,view,vigo,villega,villetaneus,vis,vocab,vorh,way,webcont,wei,wel,wellbeingmedico,western,work,workshop,workshopa,workshopsearch,workshopsem,writ,wu,wwwspringercomgpcomputersciencelncsconferenceproceedingsguidelineseasychair,wwwtechnicolorcomdreamresearchinnovationmoviememorabilitydatasettask,xavy,xiao,xintao,yadat,year,zurich,ştef
wordDict0,0.001429,0.000000,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.000000,0.000000,0.000000,0.001429,0.001429,0.001429,0.001429,0.000000,0.002857,0.00000,0.000000,0.000000,0.000000,0.001429,0.001429,0.000000,0.001429,0.001429,0.000000,0.001429,0.002857,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001429,0.031429,0.001429,0.001429,0.002857,0.000000,0.000000,0.000000,0.007143,0.000000,0.000000,0.001429,0.000000,0.000000,0.002857,0.001429,0.000000,0.002857,0.002857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001429,0.001429,0.001429,0.001429,0.001429,0.002857,0.000000,0.001429
wordDict1,0.000000,0.004942,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001647,0.001647,0.001647,0.000000,0.000000,0.000000,0.000000,0.001647,0.000000,0.00659,0.001647,0.001647,0.001647,0.000000,0.000000,0.001647,0.000000,0.000000,0.001647,0.000000,0.000000,...,0.001647,0.001647,0.001647,0.001647,0.001647,0.001647,0.003295,0.000000,0.000000,0.000000,0.000000,0.000000,0.001647,0.001647,0.001647,0.000000,0.001647,0.001647,0.000000,0.001647,0.001647,0.001647,0.000000,0.001647,0.001647,0.011532,0.001647,0.001647,0.001647,0.001647,0.001647,0.001647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001647,0.000000


,10000,1159pm,14691482,16921703,2008videos,2009videoclef,2010videoclef,2011mediaeval,2012mediaeval,2013mediaeval,2014mediaeval,2015mediaeval,2016mediaeval,2017mediaeval,2018mediaeval,2019emotion,2019runs,2019test,2019working,2020camera,2020notificationsfull,2020short,23902398,2729,27302739,2931,42nd,abl,abstract,abus,abusep,acceiv,accompany,acm,addit,address,adity,adv,advert,aesthet,...,usathierry,usayonghu,usayoshinobu,usazit,valenc,vast,venu,via,video,videomediaev,videomem,view,vigo,villega,villetaneus,vis,vocab,vorh,way,webcont,wei,wel,wellbeingmedico,western,work,workshop,workshopa,workshopsearch,workshopsem,writ,wu,wwwspringercomgpcomputersciencelncsconferenceproceedingsguidelineseasychair,wwwtechnicolorcomdreamresearchinnovationmoviememorabilitydatasettask,xavy,xiao,xintao,yadat,year,zurich,ştef
wordDict0,0.00043,0.000000,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.000000,0.000000,0.000000,0.00043,0.00043,0.00043,0.00043,0.000000,0.00086,0.000000,0.000000,0.000000,0.000000,0.00043,0.00043,0.000000,0.00043,0.00043,0.000000,0.00043,0.00086,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00043,0.009461,0.00043,0.00043,0.00086,0.000000,0.000000,0.000000,0.00215,0.000000,0.000000,0.00043,0.000000,0.000000,0.0,0.00043,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00043,0.00043,0.00043,0.00043,0.00043,0.00086,0.000000,0.00043
wordDict1,0.00000,0.001488,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000496,0.000496,0.000496,0.00000,0.00000,0.00000,0.00000,0.000496,0.00000,0.001984,0.000496,0.000496,0.000496,0.00000,0.00000,0.000496,0.00000,0.00000,0.000496,0.00000,0.00000,...,0.000496,0.000496,0.000496,0.000496,0.000496,0.000496,0.000992,0.00000,0.000000,0.00000,0.00000,0.00000,0.000496,0.000496,0.000496,0.00000,0.000496,0.000496,0.00000,0.000496,0.000496,0.0,0.00000,0.000496,0.0,0.0,0.000496,0.000496,0.000496,0.000496,0.000496,0.000496,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000496,0.00000


                           tf.idf(doc)
mem                           0.013331
video                         0.009461
predict                       0.006021
multimed                      0.004730
two thousand and eighteen     0.003870
...                                ...
mougin                        0.000000
cov                           0.000000
moreov                        0.000000
month                         0.000000
portugalheld                  0.000000

[727 rows x 1 columns]
            tf.idf(doc)
nat            0.002976
sci            0.002976
ir             0.002480
medicin        0.002480
heal           0.002480
...                 ...
icmr           0.000000
iccv           0.000000
icassp         0.000000
hyperoptim     0.000000
ştef           0.000000

[727 rows x 1 columns]


In [0]:

'''----get a list of html links/ URL's from the user that need to be processed---
'''
def get_webpages():
  html_link_list=[]
  user_input='Yes'

  while(user_input=='Yes' or user_input=='yes'):
    html_link_list.append(input("Please enter the web link : "))
    user_input=input("Do you wish to enter more html links ? (yes/NO) : ")
  print("\n In the function HTML LIST : ")
  print(html_link_list)
  return html_link_list

#Following can be uncommented to download the HTML lists. You will have a prompt to select the destination and the file name.
#download_file(html_link_list)

In [0]:
'''----Following are performed in the below code :
'''
def preprocessing(html_link):
  
  #----HTML Parsing----

  #Reading web page by calling read_web_page() function
  html=read_web_page(html_link)
  
  #Getting the plain text from the web page read above by calling get_html_text() function
  text=get_html_text(html)
  
  #----Pre-processing: Sentence Splitting, Tokenization, Normalization and removal of stopwords----

  #Split the sentence by calling sentence_splitting() function. It returns a list of strings separated by '. '.
  split=sentence_splitting(text)

  '''Tokenize the words in the sentences by using tokenize_word function. This function returns a list of lists after splitting the words in every
  sentence.
  '''
  list_token=[]
  list_token=tokenize_word(split)
  
  #Normalization
  norm_tokens = []
  for j in list_token:
    norm_token = normalize(j)
    norm_tokens.append(norm_token)
  
  #Convert list of lists to one lists
  norm_tokens_list = convert_listoflists(norm_tokens)
 
  #remove English Language stopwords - Fro example 'for', 'the', 'in' etc
  filtered_tokens = delete_stopwords(norm_tokens_list)
 
  #----lemmatization and Stemming----
  lemmas = lemmatization(filtered_tokens)
 
  stems=[]
  #Stemming
  stems = stemming(lemmas)
  return stems

In [0]:
def pos_tagging(stem_list):
    #----Part-of-Speech Tagging----
    tag = nltk.pos_tag(stem_list)
    return tag

In [0]:
#----Entity recognition----
def entity_reco(tag):
  entity_reco_list = ne_chunk(tag, binary=True)
  return entity_reco_list

In [0]:
# The function read the HTML pages and decodes them in utf-8 format. The format is usually present in the meta inormation of the web page.
def read_web_page(html_link):
  return urllib.request.urlopen(html_link).read().decode('utf8')

In [0]:
def get_html_text(html_link):
  
  soup = BeautifulSoup(html_link)
  
  '''---Delete script and style---
  This helps in removing the <script> and <style> html tag data and read that text as normal words. For example, when getting the text from the html_link
  'Media Memorability', the 'Contact' hyperlink at the end is not expanded to include the javascript as below-
  "var _rwObsfuscatedHref0 = "mai";var _rwObsfuscatedHref1 = "lto";var _rwObsfuscatedHref2 = ":m.";var _rwObsfuscatedHref3 = "a.l";
  var _rwObsfuscatedHref4 = "ars";var _rwObsfuscatedHref5 = "on@";var _rwObsfuscatedHref6 = "tud";var _rwObsfuscatedHref7 = "elf";
  var _rwObsfuscatedHref8 = "t.n";var _rwObsfuscatedHref9 = "l";  var _rwObsfuscatedHref = _rwObsfuscatedHref0+_rwObsfuscatedHref1+_rwObsfuscatedHref2+
  _rwObsfuscatedHref3+_rwObsfuscatedHref4+_rwObsfuscatedHref5+_rwObsfuscatedHref6+_rwObsfuscatedHref7+_rwObsfuscatedHref8+_rwObsfuscatedHref9; 
  document.getElementById('rw_email_contact').href = _rwObsfuscatedHref;"
  Instead it is read as normal text word 'Contact'.
  '''
  for script in soup(["script", "style"]):
    script.extract()
    
  #----Get all the text from the html_link passed to this function.----
  text = soup.get_text()
  
  #---- Remove leading and trailing spaces in each line----
  lines = (line.strip() for line in text.splitlines())
    
  #----break multi-headlines into each line----
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  
  #----remove blank lines and club the text----
  text = '\n'.join(chunk for chunk in chunks if chunk)
  
  return text

In [0]:
#----Sentence Splitting function. The tokenize function splits the text by '. ' and returns a list of these strings.----

def sentence_splitting(text):
  split = sent_tokenize(text)
  return split


In [0]:
def tokenize_word(split_list):
  list_token=[]
  for i in split_list:
    token = word_tokenize(i)
    list_token.append(token)
  return list_token

In [0]:
#Replacing numbers into characters. For example 9 becomes 'nine'.
def rep_numbers(token):
    p = inflect.engine()
    words = []
    for i in token:
        if i.isdigit():
            new_word = p.number_to_words(i)
            words.append(new_word)
        else:
            words.append(i)
    return words


#Change the all words to lowercase
def lowerc(token):
    words = []
    for i in token:
        nrm_word = i.lower()
        words.append(nrm_word)
    return words

#Remove punctuation
def punct(token):
    words = []
    for i in token:
        nrm_word = re.sub(r'[^\w\s]','', i)
        if nrm_word != '':
            words.append(nrm_word)
    return words

#Call the above three functions from one function, this function is called from the main function.
def normalize(token):
    token = rep_numbers(token)
    token = lowerc(token)
    token = punct(token)
    return token

In [0]:
def convert_listoflists(token_listoflists):
  norm_list = []
  for i in token_listoflists:
    for j in i:
        norm_list.append(j)
  return norm_list

In [0]:
def delete_stopwords(token_list):
  
  #List of English language Stopwords
  stop_words = set(stopwords.words('english'))

  #Filtering stopwords from the token list
  filtered = []
  for i in token_list:
    if i not in stop_words:
        filtered.append(i)
  return filtered

In [0]:
def lemmatization(filtered):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    #print("in lemmatization")
    for i in filtered:
        #print("In for loop")
        lemma = lemmatizer.lemmatize(i, pos='v')
        lemmas.append(lemma)
    #print("\n in lemmatization, type of lemmas returned : ", type(lemmas))
    return lemmas

In [0]:
def stemming(lemmas):
    stemmer = LancasterStemmer()
    stems = []
    for i in lemmas:
        stem = stemmer.stem(i)
        stems.append(stem)
    return stems

In [0]:
def word_count(worddict, stemlist):
  for word in stemlist:
    worddict[word]+=1
  return worddict

In [0]:
#Creating the function to calculate the TF
def computeTF (wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word]=count/float(bowCount)
    return tfDict

In [0]:
#Create the fucntion to calculate the IDF
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [0]:
#Create the function to calculate the TF.IDF score
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [0]:
def sort_score(tfidfBowval):
  score = pd.DataFrame([tfidfBowval])
  scorest = score.T
  scorest.columns = ['tf.idf(doc)']
  scorestf = scorest.sort_values(['tf.idf(doc)'], ascending = False)
  print(scorestf)
  return scorestf

In [0]:
def download_file(filename):
  filename=pd.DataFrame(filename)
  filename.to_csv('Select Name while downloading.csv')
  files.download('Select Name while downloading.csv')

In [0]:
# this is the list of all stems that were returned after stemming. Each list belong to each webpage.

def var_dictionary(html_link_list):
  stem_list=[]
  for i in range(len(html_link_list)):
    stem_list.append(preprocessing(html_link_list[i]))

  #create a set of words from the list of stems that is combine all the stem lists and sort it
  set_of_words={}
  for i in range(len(stem_list)):
    set_of_words=set(set_of_words).union(set(stem_list[i]))
  set_of_words=sorted(set_of_words)

  # Create a dictionary of dictionary type variables. The number of variables depends on the number of html links provided initially or the length of the stem list any which way.
  var_dict={}
  for x in range(len(stem_list)):
    var_dict["wordDict{0}".format(x)]={}

  #Each dictionary variable created above is assigned the set of words with 0 value so it is easy to calculate the occurrences to calculate tf-idf
  for key in var_dict.keys():
    var_dict[key]=dict.fromkeys(set_of_words,0)
  return var_dict

In [0]:
#For every dictionary belonging to every html link, call the word count function which provides the count for each word in its corresponding stem
def word_freq(d, stem_list):
  stem_list_index=0
  for key, dval in d.items():
    d[key]=word_count(dval,stem_list[stem_list_index])
    stem_list_index +=1

  #For displaying properly as a data frame
  test=pd.DataFrame(d)
  test=test.transpose()
  display(test)
  return d